In [1]:
import sqlalchemy
import pandas as pd
from sqlconfig import connection_string
import numpy as np

In [2]:
engine = sqlalchemy.create_engine(connection_string)

In [3]:
# Query our unclean data

query = '''
SELECT * FROM gpudata
'''

In [4]:
df = pd.read_sql(query, engine)

In [5]:
df.head(1)

,webscraperorder,webscraperstarturl,pagination,productname,price,imagesrc,link,linkhref,brand1,ramsize1,...,productdims,itemdimensionslwh,countryorigin,dateavailable,numreviews,rating1,interface,ramtype,rating2,rating3
0,1666589751-610,https://www.amazon.com/s?k=gpu+graphics+card&s...,https://www.amazon.com/s?k=gpu+graphics+card&p...,ASUS ROG Strix NVIDIA GeForce RTX 3080 OC Edit...,$799.99,https://m.media-amazon.com/images/I/71bda4nJ2J...,ASUS ROG Strix NVIDIA GeForce RTX 3080 OC Edit...,https://www.amazon.com/ASUS-Graphics-DisplayPo...,ASUS,12 GB,...,‎12.53 x 5.51 x 2.27 inches,‎12.53 x 5.51 x 2.27 inches,‎China,"‎January 11, 2022",143 ratings,null,PCI-Express x16,GDDR6X,null,null


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8476 entries, 0 to 8475
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   webscraperorder     8476 non-null   object
 1   webscraperstarturl  8476 non-null   object
 2   pagination          8476 non-null   object
 3   productname         8476 non-null   object
 4   price               8476 non-null   object
 5   imagesrc            8476 non-null   object
 6   link                8476 non-null   object
 7   linkhref            8476 non-null   object
 8   brand1              8476 non-null   object
 9   ramsize1            8476 non-null   object
 10  clockspeed1         8476 non-null   object
 11  clockspeed2         8476 non-null   object
 12  chipsetmanu         8476 non-null   object
 13  ramsize2            8476 non-null   object
 14  brand2              8476 non-null   object
 15  series              8476 non-null   object
 16  itemweight          8476

In [7]:
# Query columns for dataset creation
# Column explanation shown in README

In [8]:
cleaning_query = '''
SELECT productname,
       CASE
              WHEN productname ilike '%%ASUS%%' THEN 'ASUS'
              WHEN productname ilike '%%PNY%%' THEN 'PNY'
              WHEN productname ilike '%%ZOTAC%%' THEN 'ZOTAC'
              WHEN productname ilike '%%Gigabyte%%' THEN 'GIGABYTE'
              WHEN productname ilike '%%EVGA%%' THEN 'EVGA'
              WHEN productname ilike '%%MSI%%' THEN 'MSI'
              WHEN productname ilike '%%NVIDIA%%' THEN 'NVIDIA'
              WHEN productname ilike '%%Sapphire%%' THEN 'SAPPHIRE'
              WHEN productname ilike '%%ASRock%%' THEN 'ASROCK'
              WHEN productname ilike '%%PowerColor%%' THEN 'POWERCOLOR'
              WHEN productname ilike '%%VisionTek%%' THEN 'VISIONTEK'
              WHEN productname ilike '%%XFX%%' THEN 'XFX'
              WHEN productname ilike '%%PNY%%' THEN 'PNY'
              WHEN productname ilike '%%ZOTAC%%' THEN 'ZOTAC'
              WHEN productname ilike '%%DELL%%' THEN 'DELL'
              WHEN productname ilike '%%HP%%' THEN 'HP'
              WHEN productname ilike '%%YESTON%%' THEN 'YESTON'
              ELSE NULL
       end AS brand,
       ramsize1 as ramsize,
       clockspeed1 AS gpuclockspeed,
       clockspeed2 AS memoryspeed,
       CASE
              WHEN productname ilike '%%RTX%%'
              OR     productname ilike '%%GeForce%%' THEN 'NVIDIA'
              WHEN productname ilike '%%RX%%'
              OR     productname ilike '%%Radeon%%' THEN 'AMD'
              ELSE NULL
       end AS chipmanu,
       itemweight,
       productdims,
       REPLACE(numreviews, ',', '') as numreviews,
       REPLACE(REPLACE(price, '$', ''), ',', '') as price
FROM   gpudata
'''

In [9]:
preproc_data = pd.read_sql(cleaning_query, engine)

In [10]:
preproc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8476 entries, 0 to 8475
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   productname    8476 non-null   object
 1   brand          6684 non-null   object
 2   ramsize        8476 non-null   object
 3   gpuclockspeed  8476 non-null   object
 4   memoryspeed    8476 non-null   object
 5   chipmanu       6033 non-null   object
 6   itemweight     8476 non-null   object
 7   productdims    8476 non-null   object
 8   numreviews     8476 non-null   object
 9   price          8476 non-null   object
dtypes: object(10)
memory usage: 662.3+ KB


In [11]:
preproc_data.head(20)

,productname,brand,ramsize,gpuclockspeed,memoryspeed,chipmanu,itemweight,productdims,numreviews,price
0,ASUS ROG Strix NVIDIA GeForce RTX 3080 OC Edit...,ASUS,12 GB,1.89 GHz,‎4000 MHz,NVIDIA,‎3.9 pounds,‎12.53 x 5.51 x 2.27 inches,143 ratings,799.99
1,Sapphire 11310-01-20G Pulse AMD Radeon RX 6600...,SAPPHIRE,8 GB,2491 MHz,‎16 GHz,AMD,‎1.59 pounds,‎10.5 x 6.5 x 2.5 inches,212 ratings,249.99
2,ASUS NVIDIA GeForce GT 730 Graphics Card (PCIe...,ASUS,2 GB,902 MHz,‎5010 MHz,NVIDIA,‎15.2 ounces,‎5.8 x 4.1 x 0.7 inches,12 ratings,84.99
3,ASUS TUF Gaming NVIDIA GeForce RTX 3060 V2 OC ...,ASUS,12 GB,1882 MHz,‎1882 MHz,NVIDIA,‎2.4 pounds,‎11.81 x 5.63 x 2.13 inches,2 ratings,489.99
4,MSI Gaming GeForce GTX 1660 Super 192-bit HDMI...,MSI,6 GB,1785 MHz,‎14 GHz,NVIDIA,‎1.47 pounds,‎8 x 1.7 x 5 inches,1678 ratings,249.00
5,ASUS GeForce GTX 1050 Ti 4GB Phoenix Fan Editi...,ASUS,4 GB,1392 MHz,‎1392 MHz,NVIDIA,‎1.2 pounds,‎7.6 x 1.5 x 4.4 inches,3092 ratings,183.88
6,EVGA GeForce GT 730 2GB (Low Profile),EVGA,2 GB,902,0.4ns,NVIDIA,‎6.4 ounces,‎5.7 x 1.56 x 7.63 inches,715 ratings,56.00
7,ASUS ROG Strix NVIDIA GeForce RTX 3060 V2 OC E...,ASUS,12 GB,1.91 GHz,‎1912 MHz,NVIDIA,‎2.5 pounds,‎11.81 x 5.26 x 2.11 inches,711 ratings,null
8,ZOTAC Gaming GeForce GTX 1660 6GB GDDR5 192-bi...,ZOTAC,6 GB,1785 MHz,‎1785 MHz,NVIDIA,‎2.18 pounds,‎8.5 x 1.39 x 4.38 inches,2102 ratings,217.42
9,MSI Gaming GeForce RTX 2060 6GB GDRR6 192-bit ...,MSI,6 GB,1.71 GHz,‎1710 MHz,NVIDIA,‎1.68 pounds,‎9.1 x 1.7 x 5 inches,341 ratings,274.99


In [12]:
# Pre-processing number of reviews column

In [13]:
# use .split() to split each row into a list
reviews = preproc_data.numreviews.str.split(expand=True) 

# use the first element of the list to get the number of reviews for our review column
preproc_data['numreviews'] = reviews[0]

# change 'null' and 'None' into NaN and drop values
preproc_data = preproc_data.replace('null', np.nan) 
preproc_data = preproc_data.dropna()

# drop specific rows
preproc_data = preproc_data.drop([190])
preproc_data = preproc_data.drop([6])

# change data types of price and review columns 
preproc_data['price'] = preproc_data['price'].astype(float).apply(np.ceil).astype('int64')
preproc_data['numreviews'] = pd.to_numeric(preproc_data['numreviews'])

In [14]:
# use .split() and rename synthesized columns for pre-processing
gpuspeed = preproc_data.gpuclockspeed.str.split(expand=True)
gpuspeed = gpuspeed.rename(columns={0: 'unit', 1: 'measure'})
gpuspeed['unit'] = gpuspeed['unit'].astype('float64')

# synthesize new columns for dataframe operation
gpuspeed.loc[gpuspeed.measure == 'GHz', 'GHz'] = gpuspeed['unit']
gpuspeed.loc[gpuspeed.measure == 'MHz', 'MHz'] = gpuspeed['unit']

# convert GHz to MHz and vice versa
gpuspeed['g2m'] = gpuspeed[['GHz']] * 1000
gpuspeed['m2g'] = gpuspeed[['MHz']] / 1000

# combine dataframes together
gpuspeed['gpu_ghz'] = gpuspeed['GHz'].add(gpuspeed['m2g'], fill_value=0)
gpuspeed['gpu_mhz'] = gpuspeed['MHz'].add(gpuspeed['g2m'], fill_value=0)

# add synthesized columns into dataset
gpuspeed = gpuspeed[['gpu_ghz', 'gpu_mhz']]
preproc_data['gpuclockspeed_ghz'] = gpuspeed[['gpu_ghz']]
preproc_data['gpuclockspeed_mhz'] = gpuspeed[['gpu_mhz']]

In [15]:
preproc_data.head(20)

,productname,brand,ramsize,gpuclockspeed,memoryspeed,chipmanu,itemweight,productdims,numreviews,price,gpuclockspeed_ghz,gpuclockspeed_mhz
0,ASUS ROG Strix NVIDIA GeForce RTX 3080 OC Edit...,ASUS,12 GB,1.89 GHz,‎4000 MHz,NVIDIA,‎3.9 pounds,‎12.53 x 5.51 x 2.27 inches,143,800,1.890,1890.0
1,Sapphire 11310-01-20G Pulse AMD Radeon RX 6600...,SAPPHIRE,8 GB,2491 MHz,‎16 GHz,AMD,‎1.59 pounds,‎10.5 x 6.5 x 2.5 inches,212,250,2.491,2491.0
2,ASUS NVIDIA GeForce GT 730 Graphics Card (PCIe...,ASUS,2 GB,902 MHz,‎5010 MHz,NVIDIA,‎15.2 ounces,‎5.8 x 4.1 x 0.7 inches,12,85,0.902,902.0
3,ASUS TUF Gaming NVIDIA GeForce RTX 3060 V2 OC ...,ASUS,12 GB,1882 MHz,‎1882 MHz,NVIDIA,‎2.4 pounds,‎11.81 x 5.63 x 2.13 inches,2,490,1.882,1882.0
4,MSI Gaming GeForce GTX 1660 Super 192-bit HDMI...,MSI,6 GB,1785 MHz,‎14 GHz,NVIDIA,‎1.47 pounds,‎8 x 1.7 x 5 inches,1678,249,1.785,1785.0
5,ASUS GeForce GTX 1050 Ti 4GB Phoenix Fan Editi...,ASUS,4 GB,1392 MHz,‎1392 MHz,NVIDIA,‎1.2 pounds,‎7.6 x 1.5 x 4.4 inches,3092,184,1.392,1392.0
8,ZOTAC Gaming GeForce GTX 1660 6GB GDDR5 192-bi...,ZOTAC,6 GB,1785 MHz,‎1785 MHz,NVIDIA,‎2.18 pounds,‎8.5 x 1.39 x 4.38 inches,2102,218,1.785,1785.0
9,MSI Gaming GeForce RTX 2060 6GB GDRR6 192-bit ...,MSI,6 GB,1.71 GHz,‎1710 MHz,NVIDIA,‎1.68 pounds,‎9.1 x 1.7 x 5 inches,341,275,1.710,1710.0
10,ZOTAC GAMING GeForce RTX™ 3060 AMP White Editi...,ZOTAC,12 GB,1867 MHz,‎1867 MHz,NVIDIA,‎2.2 pounds,‎9.1 x 5.6 x 1.6 inches,183,400,1.867,1867.0
11,MSI Gaming GeForce RTX 3060 12GB 15 Gbps GDRR6...,MSI,12 GB,1710 MHz,‎1807 MHz,NVIDIA,‎2.22 pounds,‎9.3 x 1.7 x 4.9 inches,597,370,1.710,1710.0


In [16]:
# use .split() and rename synthesized columns for pre-processing
memspeed = preproc_data.memoryspeed.str.split(expand=True)
memspeed = memspeed.rename(columns={0: 'unit', 1: 'measure'})

# synthesize new columns for dataframe operation
memspeed = memspeed.replace('TURBO', np.nan)
memspeed = memspeed.replace('None', np.nan)

# remove characters before converting column datatype 
memspeed['unit'] = memspeed['unit'].str.removeprefix('\u200e')
memspeed['unit'] = memspeed['unit'].astype('float64')

# convert GHz to MHz and vice versa and perform dataframe operation
memspeed.loc[memspeed.measure == 'GHz', 'GHz'] = memspeed['unit']
memspeed.loc[memspeed.measure == 'MHz', 'MHz'] = memspeed['unit']
memspeed['g2m'] = memspeed[['GHz']] * 1000
memspeed['m2g'] = memspeed[['MHz']] / 1000

# merge columns into dataset
memspeed['mem_ghz'] = memspeed['GHz'].add(memspeed['m2g'], fill_value=0)
memspeed['mem_mhz'] = memspeed['MHz'].add(memspeed['g2m'], fill_value=0)
memspeed = memspeed[['mem_ghz', 'mem_mhz']]
preproc_data['memspeed_ghz'] = memspeed[['mem_ghz']]
preproc_data['memspeed_mhz'] = memspeed[['mem_mhz']]

In [17]:
preproc_data.head(20)

,productname,brand,ramsize,gpuclockspeed,memoryspeed,chipmanu,itemweight,productdims,numreviews,price,gpuclockspeed_ghz,gpuclockspeed_mhz,memspeed_ghz,memspeed_mhz
0,ASUS ROG Strix NVIDIA GeForce RTX 3080 OC Edit...,ASUS,12 GB,1.89 GHz,‎4000 MHz,NVIDIA,‎3.9 pounds,‎12.53 x 5.51 x 2.27 inches,143,800,1.890,1890.0,4.000,4000.0
1,Sapphire 11310-01-20G Pulse AMD Radeon RX 6600...,SAPPHIRE,8 GB,2491 MHz,‎16 GHz,AMD,‎1.59 pounds,‎10.5 x 6.5 x 2.5 inches,212,250,2.491,2491.0,16.000,16000.0
2,ASUS NVIDIA GeForce GT 730 Graphics Card (PCIe...,ASUS,2 GB,902 MHz,‎5010 MHz,NVIDIA,‎15.2 ounces,‎5.8 x 4.1 x 0.7 inches,12,85,0.902,902.0,5.010,5010.0
3,ASUS TUF Gaming NVIDIA GeForce RTX 3060 V2 OC ...,ASUS,12 GB,1882 MHz,‎1882 MHz,NVIDIA,‎2.4 pounds,‎11.81 x 5.63 x 2.13 inches,2,490,1.882,1882.0,1.882,1882.0
4,MSI Gaming GeForce GTX 1660 Super 192-bit HDMI...,MSI,6 GB,1785 MHz,‎14 GHz,NVIDIA,‎1.47 pounds,‎8 x 1.7 x 5 inches,1678,249,1.785,1785.0,14.000,14000.0
5,ASUS GeForce GTX 1050 Ti 4GB Phoenix Fan Editi...,ASUS,4 GB,1392 MHz,‎1392 MHz,NVIDIA,‎1.2 pounds,‎7.6 x 1.5 x 4.4 inches,3092,184,1.392,1392.0,1.392,1392.0
8,ZOTAC Gaming GeForce GTX 1660 6GB GDDR5 192-bi...,ZOTAC,6 GB,1785 MHz,‎1785 MHz,NVIDIA,‎2.18 pounds,‎8.5 x 1.39 x 4.38 inches,2102,218,1.785,1785.0,1.785,1785.0
9,MSI Gaming GeForce RTX 2060 6GB GDRR6 192-bit ...,MSI,6 GB,1.71 GHz,‎1710 MHz,NVIDIA,‎1.68 pounds,‎9.1 x 1.7 x 5 inches,341,275,1.710,1710.0,1.710,1710.0
10,ZOTAC GAMING GeForce RTX™ 3060 AMP White Editi...,ZOTAC,12 GB,1867 MHz,‎1867 MHz,NVIDIA,‎2.2 pounds,‎9.1 x 5.6 x 1.6 inches,183,400,1.867,1867.0,1.867,1867.0
11,MSI Gaming GeForce RTX 3060 12GB 15 Gbps GDRR6...,MSI,12 GB,1710 MHz,‎1807 MHz,NVIDIA,‎2.22 pounds,‎9.3 x 1.7 x 4.9 inches,597,370,1.710,1710.0,1.807,1807.0


In [18]:
# split, rename columns, and change data type 
ram = preproc_data.ramsize.str.split(expand=True)
ram = ram.rename(columns={0: 'unit', 1: 'measure'})
ram['unit'] = ram['unit'].astype('int64')

# add new column 
ram.loc[ram.measure == 'GB', 'GB'] = ram['unit']

# change four digit values (in MB) to GB for dataframe operation
ram.loc[ram['unit'] == 2048] = 2
ram.loc[ram['unit'] == 8000] = 8
ram['MB'] = ram[['unit']] * 1000

# include new column into dataset
ram = ram[['GB', 'MB']]
preproc_data[['ramsize_gb', 'ramsize_mb']] = ram

In [19]:
preproc_data.head(20)

,productname,brand,ramsize,gpuclockspeed,memoryspeed,chipmanu,itemweight,productdims,numreviews,price,gpuclockspeed_ghz,gpuclockspeed_mhz,memspeed_ghz,memspeed_mhz,ramsize_gb,ramsize_mb
0,ASUS ROG Strix NVIDIA GeForce RTX 3080 OC Edit...,ASUS,12 GB,1.89 GHz,‎4000 MHz,NVIDIA,‎3.9 pounds,‎12.53 x 5.51 x 2.27 inches,143,800,1.890,1890.0,4.000,4000.0,12,12000
1,Sapphire 11310-01-20G Pulse AMD Radeon RX 6600...,SAPPHIRE,8 GB,2491 MHz,‎16 GHz,AMD,‎1.59 pounds,‎10.5 x 6.5 x 2.5 inches,212,250,2.491,2491.0,16.000,16000.0,8,8000
2,ASUS NVIDIA GeForce GT 730 Graphics Card (PCIe...,ASUS,2 GB,902 MHz,‎5010 MHz,NVIDIA,‎15.2 ounces,‎5.8 x 4.1 x 0.7 inches,12,85,0.902,902.0,5.010,5010.0,2,2000
3,ASUS TUF Gaming NVIDIA GeForce RTX 3060 V2 OC ...,ASUS,12 GB,1882 MHz,‎1882 MHz,NVIDIA,‎2.4 pounds,‎11.81 x 5.63 x 2.13 inches,2,490,1.882,1882.0,1.882,1882.0,12,12000
4,MSI Gaming GeForce GTX 1660 Super 192-bit HDMI...,MSI,6 GB,1785 MHz,‎14 GHz,NVIDIA,‎1.47 pounds,‎8 x 1.7 x 5 inches,1678,249,1.785,1785.0,14.000,14000.0,6,6000
5,ASUS GeForce GTX 1050 Ti 4GB Phoenix Fan Editi...,ASUS,4 GB,1392 MHz,‎1392 MHz,NVIDIA,‎1.2 pounds,‎7.6 x 1.5 x 4.4 inches,3092,184,1.392,1392.0,1.392,1392.0,4,4000
8,ZOTAC Gaming GeForce GTX 1660 6GB GDDR5 192-bi...,ZOTAC,6 GB,1785 MHz,‎1785 MHz,NVIDIA,‎2.18 pounds,‎8.5 x 1.39 x 4.38 inches,2102,218,1.785,1785.0,1.785,1785.0,6,6000
9,MSI Gaming GeForce RTX 2060 6GB GDRR6 192-bit ...,MSI,6 GB,1.71 GHz,‎1710 MHz,NVIDIA,‎1.68 pounds,‎9.1 x 1.7 x 5 inches,341,275,1.710,1710.0,1.710,1710.0,6,6000
10,ZOTAC GAMING GeForce RTX™ 3060 AMP White Editi...,ZOTAC,12 GB,1867 MHz,‎1867 MHz,NVIDIA,‎2.2 pounds,‎9.1 x 5.6 x 1.6 inches,183,400,1.867,1867.0,1.867,1867.0,12,12000
11,MSI Gaming GeForce RTX 3060 12GB 15 Gbps GDRR6...,MSI,12 GB,1710 MHz,‎1807 MHz,NVIDIA,‎2.22 pounds,‎9.3 x 1.7 x 4.9 inches,597,370,1.710,1710.0,1.807,1807.0,12,12000


In [20]:
preproc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2805 entries, 0 to 8474
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   productname        2805 non-null   object 
 1   brand              2805 non-null   object 
 2   ramsize            2805 non-null   object 
 3   gpuclockspeed      2805 non-null   object 
 4   memoryspeed        2805 non-null   object 
 5   chipmanu           2805 non-null   object 
 6   itemweight         2805 non-null   object 
 7   productdims        2805 non-null   object 
 8   numreviews         2805 non-null   int64  
 9   price              2805 non-null   int64  
 10  gpuclockspeed_ghz  2805 non-null   float64
 11  gpuclockspeed_mhz  2805 non-null   float64
 12  memspeed_ghz       2783 non-null   float64
 13  memspeed_mhz       2783 non-null   float64
 14  ramsize_gb         2805 non-null   int64  
 15  ramsize_mb         2805 non-null   int64  
dtypes: float64(4), int64(4),

In [21]:
# split column, remove object characters, and convert data type
itemw = preproc_data.itemweight.str.split(expand=True)
itemw = itemw.rename(columns={0: 'unit', 1: 'measure'})
itemw['unit'] = itemw['unit'].str.removeprefix('\u200e') 
itemw['unit'] = itemw['unit'].astype('float64')

# synthesize new columns for data operations
itemw.loc[itemw.measure == 'pounds', 'pounds'] = itemw['unit']
itemw.loc[itemw.measure == 'ounces', 'ounces'] = itemw['unit']
itemw['lbs2oz'] = itemw[['pounds']] * 16
itemw['oz2lbs'] = itemw[['ounces']] / 16

itemw['iw_lbs'] = itemw['pounds'].add(itemw['oz2lbs'], fill_value=0)
itemw['iw_oz'] = itemw['ounces'].add(itemw['lbs2oz'], fill_value=0)

itemw = itemw[['iw_oz', 'iw_lbs']]

preproc_data[['iw_oz', 'iw_lbs']] = itemw

In [22]:
preproc_data.head(20)

,productname,brand,ramsize,gpuclockspeed,memoryspeed,chipmanu,itemweight,productdims,numreviews,price,gpuclockspeed_ghz,gpuclockspeed_mhz,memspeed_ghz,memspeed_mhz,ramsize_gb,ramsize_mb,iw_oz,iw_lbs
0,ASUS ROG Strix NVIDIA GeForce RTX 3080 OC Edit...,ASUS,12 GB,1.89 GHz,‎4000 MHz,NVIDIA,‎3.9 pounds,‎12.53 x 5.51 x 2.27 inches,143,800,1.890,1890.0,4.000,4000.0,12,12000,62.40,3.90000
1,Sapphire 11310-01-20G Pulse AMD Radeon RX 6600...,SAPPHIRE,8 GB,2491 MHz,‎16 GHz,AMD,‎1.59 pounds,‎10.5 x 6.5 x 2.5 inches,212,250,2.491,2491.0,16.000,16000.0,8,8000,25.44,1.59000
2,ASUS NVIDIA GeForce GT 730 Graphics Card (PCIe...,ASUS,2 GB,902 MHz,‎5010 MHz,NVIDIA,‎15.2 ounces,‎5.8 x 4.1 x 0.7 inches,12,85,0.902,902.0,5.010,5010.0,2,2000,15.20,0.95000
3,ASUS TUF Gaming NVIDIA GeForce RTX 3060 V2 OC ...,ASUS,12 GB,1882 MHz,‎1882 MHz,NVIDIA,‎2.4 pounds,‎11.81 x 5.63 x 2.13 inches,2,490,1.882,1882.0,1.882,1882.0,12,12000,38.40,2.40000
4,MSI Gaming GeForce GTX 1660 Super 192-bit HDMI...,MSI,6 GB,1785 MHz,‎14 GHz,NVIDIA,‎1.47 pounds,‎8 x 1.7 x 5 inches,1678,249,1.785,1785.0,14.000,14000.0,6,6000,23.52,1.47000
5,ASUS GeForce GTX 1050 Ti 4GB Phoenix Fan Editi...,ASUS,4 GB,1392 MHz,‎1392 MHz,NVIDIA,‎1.2 pounds,‎7.6 x 1.5 x 4.4 inches,3092,184,1.392,1392.0,1.392,1392.0,4,4000,19.20,1.20000
8,ZOTAC Gaming GeForce GTX 1660 6GB GDDR5 192-bi...,ZOTAC,6 GB,1785 MHz,‎1785 MHz,NVIDIA,‎2.18 pounds,‎8.5 x 1.39 x 4.38 inches,2102,218,1.785,1785.0,1.785,1785.0,6,6000,34.88,2.18000
9,MSI Gaming GeForce RTX 2060 6GB GDRR6 192-bit ...,MSI,6 GB,1.71 GHz,‎1710 MHz,NVIDIA,‎1.68 pounds,‎9.1 x 1.7 x 5 inches,341,275,1.710,1710.0,1.710,1710.0,6,6000,26.88,1.68000
10,ZOTAC GAMING GeForce RTX™ 3060 AMP White Editi...,ZOTAC,12 GB,1867 MHz,‎1867 MHz,NVIDIA,‎2.2 pounds,‎9.1 x 5.6 x 1.6 inches,183,400,1.867,1867.0,1.867,1867.0,12,12000,35.20,2.20000
11,MSI Gaming GeForce RTX 3060 12GB 15 Gbps GDRR6...,MSI,12 GB,1710 MHz,‎1807 MHz,NVIDIA,‎2.22 pounds,‎9.3 x 1.7 x 4.9 inches,597,370,1.710,1710.0,1.807,1807.0,12,12000,35.52,2.22000


In [23]:
# split the columns into a list and choose accordingly
dims = preproc_data.productdims.str.split(expand=True)
dims[0] = dims[0].str.removeprefix('\u200e') 
dims[2] = dims[2].str.removeprefix('\u200e') 
dims[4] = dims[4].str.removeprefix('\u200e')

# rename columns and change data types
dims = dims.rename(columns={0: 'length', 2: 'width', 4: 'height'})
dims[['length', 'width', 'height']] = dims[['length', 'width', 'height']].astype('float64')

# merge the columns into the dataset
dims = dims[['length', 'width', 'height']]
preproc_data[['length', 'width', 'height']] = dims

In [24]:
preproc_data

,productname,brand,ramsize,gpuclockspeed,memoryspeed,chipmanu,itemweight,productdims,numreviews,price,...,gpuclockspeed_mhz,memspeed_ghz,memspeed_mhz,ramsize_gb,ramsize_mb,iw_oz,iw_lbs,length,width,height
0,ASUS ROG Strix NVIDIA GeForce RTX 3080 OC Edit...,ASUS,12 GB,1.89 GHz,‎4000 MHz,NVIDIA,‎3.9 pounds,‎12.53 x 5.51 x 2.27 inches,143,800,...,1890.0,4.000,4000.0,12,12000,62.40,3.900,12.53,5.51,2.27
1,Sapphire 11310-01-20G Pulse AMD Radeon RX 6600...,SAPPHIRE,8 GB,2491 MHz,‎16 GHz,AMD,‎1.59 pounds,‎10.5 x 6.5 x 2.5 inches,212,250,...,2491.0,16.000,16000.0,8,8000,25.44,1.590,10.50,6.50,2.50
2,ASUS NVIDIA GeForce GT 730 Graphics Card (PCIe...,ASUS,2 GB,902 MHz,‎5010 MHz,NVIDIA,‎15.2 ounces,‎5.8 x 4.1 x 0.7 inches,12,85,...,902.0,5.010,5010.0,2,2000,15.20,0.950,5.80,4.10,0.70
3,ASUS TUF Gaming NVIDIA GeForce RTX 3060 V2 OC ...,ASUS,12 GB,1882 MHz,‎1882 MHz,NVIDIA,‎2.4 pounds,‎11.81 x 5.63 x 2.13 inches,2,490,...,1882.0,1.882,1882.0,12,12000,38.40,2.400,11.81,5.63,2.13
4,MSI Gaming GeForce GTX 1660 Super 192-bit HDMI...,MSI,6 GB,1785 MHz,‎14 GHz,NVIDIA,‎1.47 pounds,‎8 x 1.7 x 5 inches,1678,249,...,1785.0,14.000,14000.0,6,6000,23.52,1.470,8.00,1.70,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8466,MSI Gaming GeForce RTX 3070 LHR 8GB GDRR6 256-...,MSI,8 GB,1.85 GHz,‎1845 MHz,NVIDIA,‎3.39 pounds,‎12.7 x 2.2 x 4.7 inches,228,585,...,1850.0,1.845,1845.0,8,8000,54.24,3.390,12.70,2.20,4.70
8470,MSI Gaming GeForce RTX 3080 10GB GDRR6X 320-Bi...,MSI,10 GB,1710 MHz,‎1.71 GHz,NVIDIA,‎4.7 pounds,‎12.7 x 2.2 x 5.1 inches,303,980,...,1710.0,1.710,1710.0,10,10000,75.20,4.700,12.70,2.20,5.10
8472,PNY GeForce GT 1030 2GB Graphic Card (VCGGT103...,PNY,2 GB,1151 MHz,‎1227 MHz,NVIDIA,‎8.4 ounces,‎23.62 x 23.62 x 33.46 inches,77,110,...,1151.0,1.227,1227.0,2,2000,8.40,0.525,23.62,23.62,33.46
8473,ASUS ROG STRIX NVIDIA GeForce RTX 3060 Ti OC E...,ASUS,8 GB,189 MHz,‎40000 MHz,NVIDIA,‎2.2 pounds,‎16 x 9.2 x 3.5 inches,26,635,...,189.0,40.000,40000.0,8,8000,35.20,2.200,16.00,9.20,3.50


In [25]:
preproc_data = preproc_data.dropna()
preproc_data = preproc_data.drop(columns=['memoryspeed', 'ramsize', 'gpuclockspeed', 'productdims'])

In [26]:
preproc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2783 entries, 0 to 8474
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   productname        2783 non-null   object 
 1   brand              2783 non-null   object 
 2   chipmanu           2783 non-null   object 
 3   itemweight         2783 non-null   object 
 4   numreviews         2783 non-null   int64  
 5   price              2783 non-null   int64  
 6   gpuclockspeed_ghz  2783 non-null   float64
 7   gpuclockspeed_mhz  2783 non-null   float64
 8   memspeed_ghz       2783 non-null   float64
 9   memspeed_mhz       2783 non-null   float64
 10  ramsize_gb         2783 non-null   int64  
 11  ramsize_mb         2783 non-null   int64  
 12  iw_oz              2783 non-null   float64
 13  iw_lbs             2783 non-null   float64
 14  length             2783 non-null   float64
 15  width              2783 non-null   float64
 16  height             2783 

In [27]:
# re arrange columns 
preproc_data = preproc_data[['productname', 'brand', 'chipmanu', 'gpuclockspeed_ghz', 'gpuclockspeed_mhz', 'memspeed_ghz', 'memspeed_mhz', 'ramsize_gb', 'ramsize_mb', 'iw_oz', 'iw_lbs', 'length', 'width', 'height', 'numreviews', 'price']]

In [28]:
preproc_data.head(20)

,productname,brand,chipmanu,gpuclockspeed_ghz,gpuclockspeed_mhz,memspeed_ghz,memspeed_mhz,ramsize_gb,ramsize_mb,iw_oz,iw_lbs,length,width,height,numreviews,price
0,ASUS ROG Strix NVIDIA GeForce RTX 3080 OC Edit...,ASUS,NVIDIA,1.890,1890.0,4.000,4000.0,12,12000,62.40,3.90000,12.53,5.51,2.27,143,800
1,Sapphire 11310-01-20G Pulse AMD Radeon RX 6600...,SAPPHIRE,AMD,2.491,2491.0,16.000,16000.0,8,8000,25.44,1.59000,10.50,6.50,2.50,212,250
2,ASUS NVIDIA GeForce GT 730 Graphics Card (PCIe...,ASUS,NVIDIA,0.902,902.0,5.010,5010.0,2,2000,15.20,0.95000,5.80,4.10,0.70,12,85
3,ASUS TUF Gaming NVIDIA GeForce RTX 3060 V2 OC ...,ASUS,NVIDIA,1.882,1882.0,1.882,1882.0,12,12000,38.40,2.40000,11.81,5.63,2.13,2,490
4,MSI Gaming GeForce GTX 1660 Super 192-bit HDMI...,MSI,NVIDIA,1.785,1785.0,14.000,14000.0,6,6000,23.52,1.47000,8.00,1.70,5.00,1678,249
5,ASUS GeForce GTX 1050 Ti 4GB Phoenix Fan Editi...,ASUS,NVIDIA,1.392,1392.0,1.392,1392.0,4,4000,19.20,1.20000,7.60,1.50,4.40,3092,184
8,ZOTAC Gaming GeForce GTX 1660 6GB GDDR5 192-bi...,ZOTAC,NVIDIA,1.785,1785.0,1.785,1785.0,6,6000,34.88,2.18000,8.50,1.39,4.38,2102,218
9,MSI Gaming GeForce RTX 2060 6GB GDRR6 192-bit ...,MSI,NVIDIA,1.710,1710.0,1.710,1710.0,6,6000,26.88,1.68000,9.10,1.70,5.00,341,275
10,ZOTAC GAMING GeForce RTX™ 3060 AMP White Editi...,ZOTAC,NVIDIA,1.867,1867.0,1.867,1867.0,12,12000,35.20,2.20000,9.10,5.60,1.60,183,400
11,MSI Gaming GeForce RTX 3060 12GB 15 Gbps GDRR6...,MSI,NVIDIA,1.710,1710.0,1.807,1807.0,12,12000,35.52,2.22000,9.30,1.70,4.90,597,370


In [29]:
# preproc_data.to_csv(path_or_buf='gpu_data.csv')